In [ ]:
!pip install -q torch==2.1.0 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install -q "transformers==4.34.0" "datasets==2.13.0" "peft==0.6.2" "accelerate==0.21.0" "bitsandbytes==0.41.2.post2" "trl==0.7.9" "safetensors>=0.3.1" "gdown==4.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
from google.colab import userdata

In [ ]:
login(token=userdata.get('niru_hf_read'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model = "nirusanan/Mistral_7B_SFT_Finetune_Pandas_Code_Gen_Final"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
!gdown --id 1f4XUE4ZdZ30_SC_BzNOlyh_nf2_h1ese

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1f4XUE4ZdZ30_SC_BzNOlyh_nf2_h1ese
To: /content/pandas_code_DPO_dataset.csv
100% 33.8k/33.8k [00:00<00:00, 96.9MB/s]


In [ ]:
import pandas as pd
dataset_df = pd.read_csv('pandas_code_DPO_dataset.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
df.head()

,Dataset Name,Columns with Sample Data,Question,Error Code,Corrected Code
0,flights,"data = {\n 'Customer': ['BU79786', 'QZ44356...",effective date between 2011-04-01 to 2011-05-0...,```\ndef process(df):\n # Convert the 'Effe...,```\ndef process(df):\n from datetime impor...
1,outlet,"data = {\n 'Item_Identifier': ['FDA15', 'DR...",Total item outlet sales?,```\ndef process(df):\n # Group by 'Item_Id...,```\ndef process(df):\n # Sum up the Item_O...
2,course,"data = {\n 'course_id': [1070968, 1113822, ...",top 5 courses has high price?,```\ndef process(df):\n # Filter for course...,```\ndef process(df):\n # Sort the DataFram...
3,outlet,"data = {\n 'Item_Identifier': ['FDA15', 'DR...",Top 5 item types has high visibility?,```\ndef process(df):\n # Group by 'Item_Ty...,```\ndef process(df):\n # Group by 'Item_Ty...
4,airline,"data = {\n 'Passenger ID': [10856, 43872, 4...",Last two month how many flights were delayed?,```\ndef process(df):\n # Convert the 'Depa...,```\ndef process(df):\n from datetime impor...


In [ ]:
from datasets import Dataset

dpo_dataset = Dataset.from_pandas(df)
dpo_dataset

Dataset({
    features: ['Dataset Name', 'Columns with Sample Data', 'Question', 'Error Code', 'Corrected Code'],
    num_rows: 14
})

In [ ]:
def return_prompt_and_responses(samples):
  question = samples['Question']
  header_with_sample_data = samples['Columns with Sample Data']
  chosen_code = samples["Corrected Code"]
  error_code = samples["Error Code"]

  prompt = f"""
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
{question}

header columns with sample data:
{header_with_sample_data}

### Response:
"""

  return {
        "prompt": prompt,
        "chosen": chosen_code,
        "rejected": error_code,
    }

dataset = dpo_dataset.map(
    return_prompt_and_responses,
    remove_columns=dpo_dataset.column_names
)

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [ ]:
print(dataset[0]['prompt'])


Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
effective date between 2011-04-01 to 2011-05-01 how many customers are there?

header columns with sample data:
data = {
    'Customer': ['BU79786', 'QZ44356', 'AI49188', 'WW63253', 'HB64268'],
    'State': ['Washington', 'Arizona', 'Nevada', 'California', 'Washington'],
    'Customer Lifetime Value': [2763.519279, 6979.535903, 12887.43165, 7645.861827, 2813.692575],
    'Response': ['No', 'No', 'No', 'No', 'No'],
    'Coverage': ['Basic', 'Extended', 'Premium', 'Basic', 'Basic'],
    'Education': ['Bachelor', 'Bachelor', 'Bachelor', 'Bachelor', 'Bachelor'],
    'Effective To Date': ['2/24/11', '1/31/11', '2/19/11', '1/20/11', '2/3/11'],
    'EmploymentStatus': ['Employed', 'Unemployed', 'Employed', 'Unemployed', 'Employed'],
    'Gender': ['F', 'F', 'F', 'M', 'M'],
    'Income': [56274, 0, 48767, 0, 43836],
    'Location Code': ['Suburban', 'Su

In [ ]:
print(dataset[0]['chosen'])

```
def process(df):
    # Filter for medium-sized outlets
    medium_outlets = df[df['Outlet_Size'] == 'Medium']

    # Sum up the Item_Outlet_Sales for these outlets
    total_sales_medium_outlets = medium_outlets['Item_Outlet_Sales'].sum()

    # Return a proper message
    if total_sales_medium_outlets == 0:
        return "No sales occurred in medium-sized outlets."
    else:
        return f"The total sales in medium-sized outlets amounted to {total_sales_medium_outlets}."
```


In [ ]:
print(dataset[0]['rejected'])

```
def process(df):
    # Filter the dataframe for rows where Item_Type is 'Dairy' and Outlet_Location_Type is 'Tier 1'
    dairy_in_tier_1_outlets = df[(df['Item_Type'] == 'Dairy') & (df['Outlet_Location_Type'] == 'Tier 1')]

    # Sum the Item_Outlet_Sales for the filtered rows
    total_sales_dairy_tier_1_outlets = dairy_in_tier_1_outlets['Item_Outlet_Sales'].sum()

    return f"The total sales of dairy products in Tier 1 outlets is ${total_sales_dairy_tier_1_outlets}."
```


In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [ ]:
from transformers import TrainingArguments


training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=50,
    save_strategy="no",
    save_steps=10,
    logging_steps=1,
    output_dir="new_model",
    optim="paged_adamw_32bit",
    warmup_steps=100,
    report_to="tensorboard"

)

In [ ]:
from trl import DPOTrainer

dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=850,
    max_length=850,
)

dpo_trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:316: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.692700
4,0.688800
5,0.695200
6,0.679700
7,0.637900
8,0.646600
9,0.563500
10,0.561700


TrainOutput(global_step=50, training_loss=0.31137239876843525, metrics={'train_runtime': 768.3306, 'train_samples_per_second': 0.13, 'train_steps_per_second': 0.065, 'total_flos': 0.0, 'train_loss': 0.31137239876843525, 'epoch': 7.14})

In [ ]:
login(token=userdata.get('niru_hf_write'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
dpo_trainer.save_model("Mistral_7B_DPO_Finetune_Pandas_Code_Gen_Final")

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/Mistral_7B_DPO_Finetune_Pandas_Code_Gen_Final",
    low_cpu_mem_usage=True,
)

merged_model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
total_params = sum(p.numel() for p in merged_model.parameters())
total_params_billion = total_params / 1e9
print(f"Total parameters: {total_params_billion:.2f} Billion")

Total parameters: 7.24 Billion


In [ ]:
repo_name = "Mistral_7B_DPO_Finetune_Pandas_Code_Gen_Final"

merged_model.push_to_hub(repo_name, private=True, safe_serialization=True)
tokenizer.push_to_hub(repo_name, private=True)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/9.08G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nirusanan/Mistral_7B_DPO_Finetune_Pandas_Code_Gen_Final/commit/346efb1b7dd6f5bf4ba9e3ebcece7fdda22341e9', commit_message='Upload tokenizer', commit_description='', oid='346efb1b7dd6f5bf4ba9e3ebcece7fdda22341e9', pr_url=None, pr_revision=None, pr_num=None)